In [1]:
#General imports needed for path
import os 
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import itertools

import warnings
warnings.filterwarnings('ignore')

In [3]:
#Packages created by our group:
import route_dynamics.route_elevation.base_df as base
import route_dynamics.route_energy.knn as knn
from route_dynamics.route_riders import route_riders as ride
import route_dynamics.route_energy.longi_dynam_model as ldm

In [19]:
#User defines what routes they want to evaluate
rt_list = [22,101,102,143,150,153,154,156,157,158,159,168,169,177,178,179,180,181,182,183,186,187,190,192]

#Import route shapefile
shapefile_name1 = '../data/rt' + str(rt_list[0]) + '_pts.shp'

shapefile_name = '../data/rt' + str(rt_list[0]) + '_pts2.shp'

#Import bus stops shapefile
routes_shp = '../data/Transit_Routes_for_King_County_Metro__transitroute_line.shp'

stops_shp = '../data/Transit_Stops_for_King_County_Metro__transitstop_point.shp'

signals_shp = '../data/traffic_signals2.shp'

streets = '../data/Metro_Transportation_Network_(TNET)_in_King_County_for_Car_Mode___trans_network_car_line.shp'

trip183 = pd.read_csv("../data/Trip183.csv", usecols = ['SignRt', 'InOut', 'KeyTrip', 'BusType', 'Seats', 
                     'Period', 'AnnRides']) # KCM Data
trip183unsum = pd.read_csv("../data/Zon183Unsum.csv", usecols = ['Route', 'Dir', 'Trip_ID', 'InOut', 'STOP_SEQ', 'STOP_ID',
                     'Period', 'AveOn', 'AveOff', 'AveLd', 'Obs'])

#Acceleration profile
a_prof = pd.read_csv("../data/acceleration.csv", names=['time (s)', 'accel. (g)'])

In [20]:
option1 = gpd.read_file(shapefile_name1)
option2 = gpd.read_file(shapefile_name)

In [21]:
len(option1)

9719

In [22]:
len(option2)

9719

In [5]:
trip_df, riders, mass_bus = ride.route_ridership('PM', rt_list[0])
stop_coord, rider_coord = ride.stop_coord(rt_list[0], riders)
stop_coords = stop_coord.coordinates.values
mass_passengers = rider_coord['Mean'].values

In [11]:
len(mass_passengers + mass_bus)


56

In [6]:
signals = gpd.read_file(signals_shp)
signals = signals[signals['Route_Num']==str(rt_list[0])]
geometry = signals.geometry.values
xy = []
for i in range(len(geometry)):
    dic = mapping(geometry[i])
    coords = dic['coordinates']
    xy.append(coords)
    xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
    xy_df['coordinates'] = xy
    
xy_df

signal_coords = xy_df.coordinates.values

In [7]:
ex = ldm.RouteTrajectory(
rt_list[0],
shapefile_name,
stop_coords,
signal_coords,
mass_array=mass_bus + mass_passengers,
a_prof=a_prof
)

IllegalArgumentError: Algorithm for mass calculation must be 'list_per_stop'